In [20]:
from youtube_module import YoutubeBulider
# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')
# API_KEY 할당
API_KEY = KEY_FILE.readline()

# YoutbeModule 사용
bulider = YoutubeBulider(API_KEY)
bulider.get_videos('snippet', 'id')

AttributeError: 'NoneType' object has no attribute 'videos'